In [1]:
GPUS = [0]

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, GPUS))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from tensorflow.keras import optimizers, metrics, losses

In [3]:
from adversarial.attacks import random_attack
from adversarial.utils import dataloader
from adversarial.defences import EnsembleModel
from adversarial import models

In [4]:
BATCH_SIZE = 16
EPSILONS = np.arange(0.01, 0.5, step=0.02)
ETA = 1e-4
EPOCHS = 30

In [5]:
train_dloader = dataloader.DataLoader("data", BATCH_SIZE, training=True)
test_dloader = dataloader.DataLoader("data", BATCH_SIZE, training=False)

In [6]:
target_model = models.TargetModel()
target_model.load_custom_weights_for_mobilenet("model/model_20200507_9_1.00_0.0088")

In [7]:
ensemble_model = EnsembleModel([
    models.MobileNetV2,
    models.MobileNet,
    models.VGG16,
    models.VGG19,
    models.ResNet50,
    models.ResNet50V2
])

In [8]:
criterion = losses.SparseCategoricalCrossentropy()

train_losses = metrics.SparseCategoricalCrossentropy()
train_metrics = metrics.SparseCategoricalAccuracy()

test_losses = metrics.SparseCategoricalCrossentropy()
test_metrics = metrics.SparseCategoricalAccuracy()

optimizer = optimizers.Adam(learning_rate=ETA)

In [9]:
def train_step(images, labels, adv_labels):
    adv, _, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    
    
    n = tf.shape(adv)[0]
    slicing = tf.concat([tf.ones((n,), dtype=tf.int32), tf.cast(success, tf.int32)], axis=0)
    
    x_batch, y_batch = dataloader.concat_data(images, labels, adv)
    x_batch = x_batch[slicing == 1]
    y_batch = y_batch[slicing == 1]
    
#     for i in range(tf.shape(x_batch)[0]):
#         print(y_batch[i].numpy())
#         plt.imshow(x_batch.numpy()[i])
#         plt.show()
        
    with tf.GradientTape() as tape:
        outputs, _ = target_model(x_batch, training=True)
        loss = criterion(y_batch, outputs)
        
    train_losses.update_state(y_batch, outputs)
    train_metrics.update_state(y_batch, outputs)
        
    grads = tape.gradient(loss, target_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, target_model.trainable_variables))

In [10]:
def test_step(images, labels, adv_labels):
    adv, _, success = random_attack(ensemble_model, images, adv_labels, EPSILONS)
    
    n = tf.shape(adv)[0]
    slicing = tf.concat([tf.ones((n,), dtype=tf.int32), tf.cast(success, tf.int32)], axis=0)
    
    x_batch, y_batch = dataloader.concat_data(images, labels, adv)
    x_batch = x_batch[slicing == 1]
    y_batch = y_batch[slicing == 1]

    outputs, _ = target_model(x_batch, training=False)

    test_losses.update_state(y_batch, outputs)
    test_metrics.update_state(y_batch, outputs)

In [11]:
def train():

    train_loss_list = []
    test_loss_list = []
    
    best_test_loss = 1.0

    for e in range(EPOCHS):
        for x_batch, y_batch, adv_batch in iter(train_dloader):
            train_step(x_batch, y_batch, adv_batch)

        for x_batch, y_batch, adv_batch in iter(test_dloader):
            test_step(x_batch, y_batch, adv_batch)

        train_loss = train_losses.result()
        train_acc = train_metrics.result()

        test_loss = test_losses.result()
        test_acc = test_metrics.result()

        train_loss_list.append(train_loss)
        test_loss_list.append(test_loss)

        train_losses.reset_states()
        train_metrics.reset_states()
        test_losses.reset_states()
        test_metrics.reset_states()

        print(f"Epochs {e+1}/{EPOCHS}, train loss: {train_loss:.8f}, train acc: {train_acc:.4f}, test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")
        
        if best_test_loss >= test_loss:
            best_test_loss = test_loss
            now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            target_model.save_custom_weights(f"model/ensemble_denoising-{now}.pkl")
        
    return train_loss_list, test_loss_list


In [12]:
train_loss_list, test_loss_list = train()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Epochs 1/30, train loss: 0.45553976, train acc: 0.8685, test loss: 0.68773121, test acc: 0.8320
Epochs 2/30, train loss: 0.32827544, train acc: 0.8951, test loss: 0.73945963, test acc: 0.8193
Epochs 3/30, train loss: 0.29181796, train acc: 0.9080, test loss: 0.72667569, test acc: 0.8722
Epochs 4/30, train loss: 0.29071176, train acc: 0.9059, test loss: 0.78561556, test acc: 0.8205
Epochs 5/30, train loss: 0.28742069, train acc: 0.9082, test loss: 0.68746173, test acc: 0.8349
Epochs 6/30, train loss: 0.27721882, train acc: 0.9138, test loss: 0.62360686, test acc: 0.8417
Epochs 7/30, train loss: 0.27611762, train acc: 0.9122, test loss: 0.61586708, test acc: 0.8449
Epochs 8/30, train loss: 0.27348956, train acc: 0.9121, test loss: 0.94508946, test acc: 0.8118
Epochs 9/30, train loss: 0.26502502, train acc: 0.9156, test loss: 0.86633426, test acc: 0.7919
Epochs 10/30, train loss: 0.25907007, train acc: 0.9161